# Formparser Table to Entity Converter Tool

* Author: docai-incubator@google.com

## Disclaimer

This tool is not supported by the Google engineering team or product team. It is provided and supported on a best-effort basis by the **DocAI Incubator Team**. No guarantees of performance are implied.

## Objective

This document provides a step-by-step guide on how to use the Formparser Table to Entity Converter Tool. The tool converts Formparser tables output to entity-annotated JSON files. The user inputs a dictionary of header names and their corresponding entity names, and the tool uses fuzzy matching to map the headers to the entities. The output JSON files can be used to train and visualize entities

## Prerequisites 
* Knowledge of Python
* Python : Jupyter notebook (Vertex) or Google Colab 
* Access to Json Files in the Google Bucket

## Step by step procedure

### Download and install the required libraries

In [ ]:
!pip install fuzzywuzzy pandas google-cloud-storage google-cloud-documentai
!wget https://raw.githubusercontent.com/GoogleCloudPlatform/document-ai-samples/main/incubator-tools/best-practices/utilities/utilities.py

### Import the required Libraries

In [17]:
import json
from typing import List, Dict, Any
import pandas as pd
from fuzzywuzzy import process
from google.cloud import storage
from google.cloud import documentai_v1beta3 as documentai
import utilities
from typing import Dict, List, Optional, Tuple, TypedDict, Any, Union

### Setup the required inputs

In [15]:
user_input = {
    "ItemCode": "item code",
    "Quantity": "QTY CASE",
    "TotalPrice": "Unit Price",
    "UnitPrice": "Amount",
}
# Specify your bucket and prefix (folder)
input_bucket_name = "xxxxxxxx"
input_prefix = "xxxxxxxx/xxxxxx/xxxxxx/"

output_bucket_name = "xxxxxxxx"
output_prefix = "xxxxxx/xxxxxxxx/"

When setting up or modifying the **`user_input`** dictionary, ensure to:

Use the appropriate entity name (from your schema) as the key.
Match it with the correct header name (from the PDF) as its value.

For Example, in the **`user_input dictionary`**:
**`"ItemCode"`** is an entity name used in a schema.
**`"item code"`** is the header name that you would look for in a PDF.

**Note:** If you wish to modify the Parent Entity Name, simply replace **`"invoiceItem"`** in the code with the desired name based on your requirements.

**`input_bucket_name`** and **`output_bucket_name`** variables indicate the Google Cloud Storage bucket name. \
**`input_prefix`** denotes the directory path within the GCS bucket where input JSON files reside. \
**`output_prefix`** marks the directory path within the GCS bucket where processed or output JSON files will be stored.


### Run the required Functions

In [21]:
def text_anchor_to_text(
    document, text_anchor, page_number
) -> Dict[str, Optional[Union[str, Any]]]:
    """
    Extracts text and corresponding bounding box information from a document based on text anchors.

    Args:
    document (Document): A dictionary representing the document, containing the full text.
    text_anchor (TextAnchor): A dictionary representing the text anchor, containing text segments.
    page_number (int): The page number where the text anchor is located.

    Returns:
    Dict[str, Optional[Union[str, BoundingBox]]]: A dictionary containing the extracted text and the bounding box.
    The bounding box is represented as a dictionary with 'topLeft' and 'bottomRight' keys, each containing
    a dictionary with 'x' and 'y' coordinates. If no bounding box is found, the value is None.
    """
    response = ""
    text_segments = text_anchor.text_segments if text_anchor else []

    all_bounding_boxes = []
    for segment in text_segments:
        start_index = segment.start_index if segment.start_index else 0
        end_index = segment.end_index if segment.end_index else len(document.text)

        bounding_box, _, _ = get_token(
            document,
            page_number,
            [{"start_index": str(start_index), "end_index": str(end_index)}],
        )
        vertices = {
            "topLeft": {"x": bounding_box["min_x"], "y": bounding_box["min_y"]},
            "bottomRight": {"x": bounding_box["max_x"], "y": bounding_box["max_y"]},
        }

        if vertices:
            response += document.text[start_index:end_index]
            all_bounding_boxes.append(vertices)

    if all(box is None for box in all_bounding_boxes):
        return {"text": response.strip().replace("\n", " "), "bounding_box": None}

    # Get the min and max values, or use defaults if the lists are empty
    min_x_list = [
        box["topLeft"]["x"]
        for box in all_bounding_boxes
        if box["topLeft"]["x"] is not None
    ]
    min_y_list = [
        box["topLeft"]["y"]
        for box in all_bounding_boxes
        if box["topLeft"]["y"] is not None
    ]
    max_x_list = [
        box["bottomRight"]["x"]
        for box in all_bounding_boxes
        if box["bottomRight"]["x"] is not None
    ]
    max_y_list = [
        box["bottomRight"]["y"]
        for box in all_bounding_boxes
        if box["bottomRight"]["y"] is not None
    ]

    if not (min_x_list and min_y_list and max_x_list and max_y_list):
        return {"text": response.strip().replace("\n", " "), "bounding_box": None}
    min_x = min(min_x_list)
    min_y = min(min_y_list)
    max_x = max(max_x_list)
    max_y = max(max_y_list)

    page_anchor = {
        "topLeft": {"x": min_x, "y": min_y},
        "bottomRight": {"x": max_x, "y": max_y},
    }

    return {
        "text": response.strip().replace("\n", " "),
        "page_anchor": page_anchor,
        "text_anchor": text_anchor,
    }


def get_token(document, page_num, text_anchors_check) -> Tuple[Any, List[Dict], float]:
    """
    Extracts the bounding box, text anchor tokens, and confidence level for specified text anchors in a document.

    Args:
    document (Document): A dictionary representing the document, containing pages with tokens.
    page_num (int): The page number to search for tokens.
    text_anchors_check (List[TextAnchorCheck]): A list of dictionaries containing 'start_index' and 'end_index'
    for text anchors to be checked.

    Returns:
    Tuple[BoundingBox, List[Dict], float]: A tuple containing the bounding box of the text (if found),
    a list of text anchor tokens, and the highest confidence level among the tokens.
    The bounding box is a dictionary with 'min_x', 'min_y', 'max_x', 'max_y'. If no bounding box is found, values are None.
    """
    min_x = min_y = max_x = max_y = None
    text_anc_token = []
    confidence = 0.0

    for page in document.pages:
        if page.page_number - 1 == page_num:
            for token in page.tokens:
                vertices = token.layout.bounding_poly.normalized_vertices
                min_x_token = min(vertex.x for vertex in vertices)
                min_y_token = min(vertex.y for vertex in vertices)
                max_x_token = max(vertex.x for vertex in vertices)
                max_y_token = max(vertex.y for vertex in vertices)

                start_index = token.layout.text_anchor.text_segments[0].start_index
                end_index = token.layout.text_anchor.text_segments[0].end_index

                # Adjusting the logic to match the text anchors
                for text_anchor_check in text_anchors_check:
                    start_index_check = int(text_anchor_check["start_index"])
                    end_index_check = int(text_anchor_check["end_index"])

                    if (
                        start_index <= start_index_check
                        and end_index >= end_index_check
                    ):
                        min_x = (
                            min_x_token if min_x is None else min(min_x, min_x_token)
                        )
                        min_y = (
                            min_y_token if min_y is None else min(min_y, min_y_token)
                        )
                        max_x = (
                            max_x_token if max_x is None else max(max_x, max_x_token)
                        )
                        max_y = (
                            max_y_token if max_y is None else max(max_y, max_y_token)
                        )
                        text_anc_token.append(token.layout.text_anchor.text_segments)
                        confidence = max(confidence, token.layout.confidence)

    return (
        {"min_x": min_x, "min_y": min_y, "max_x": max_x, "max_y": max_y},
        text_anc_token,
        confidence,
    )


def get_table_data(document, rows, page_number) -> List[List[Dict[str, Any]]]:
    """
    Extracts text data and bounding boxes from table rows in a Document AI object.

    Args:
    document (Dict[str, Any]): The Document AI object, representing the processed document.
    rows (List[Dict[str, Any]]): A list of row objects extracted from a table in the document.
    page_number (int): The page number where the table is located.

    Returns:
    List[List[Dict[str, Any]]]: A nested list where each sublist represents a row in the table.
    Each element in the sublist is a dictionary containing the text data and its corresponding bounding box
    for a cell in the row.
    """
    all_values = []
    for row in rows:
        current_row_values = []
        for cell in row.cells:
            cell_data = text_anchor_to_text(
                document, cell.layout.text_anchor, page_number
            )
            current_row_values.append(cell_data)
        all_values.append(current_row_values)
    return all_values


def read_json_from_gcs(bucket_name: str, blob_name: str) -> Dict:
    """
    Reads a JSON file from Google Cloud Storage (GCS) and returns its contents.

    Args:
    bucket_name (str): The name of the GCS bucket.
    blob_name (str): The name of the blob (file) in the GCS bucket.

    Returns:
    Dict: The contents of the JSON file as a dictionary.
    """
    bucket = client.get_bucket(bucket_name)
    blob = bucket.blob(blob_name)
    json_data = json.loads(blob.download_as_text())
    return json_data

### Execute the code

In [ ]:
# Set up the Google Cloud Storage client
client = storage.Client()

# List all .json files in the input GCS bucket with the given prefix
blobs = client.list_blobs(input_bucket_name, prefix=input_prefix)
json_files = [blob.name for blob in blobs if blob.name.endswith(".json")]


for json_file in json_files:
    print(f"Processing: {json_file}")
    json_data = read_json_from_gcs(input_bucket_name, json_file)
    if "entities" not in json_data:
        json_data["entities"] = []
    result = []
    json_string = json.dumps(json_data)
    document = documentai.Document.from_json(json_string)
    # print(document.entities)
    for page_index, page in enumerate(document.pages):
        page_number = page_index
        for table in page.tables:
            # Convert RepeatedComposite to Python lists and concatenate
            all_rows = list(table.header_rows) + list(table.body_rows)

            # Extract cell values from rows
            table_data = []
            for row in all_rows:
                row_data = get_table_data(document, [row], page_number)
                table_data.append(row_data[0])

            df = pd.DataFrame(data=table_data)
            df.index = df.index + 1
            df = df.sort_index()

            # display(df)

            if df.shape[1] > 7:
                first_row = df.iloc[0]
                actual_headers = [elem["text"] for elem in first_row]

                # Update the dataframe's columns to the actual headers
                df.columns = actual_headers

                # Mapping the user input columns to actual headers
                matched_headers = {}
                for friendly_name, input_header in user_input.items():
                    best_match, score = process.extractOne(input_header, actual_headers)
                    if score >= 70:  # Adjust the threshold if needed
                        matched_headers[friendly_name] = best_match
                    else:
                        print(f"No match found for '{input_header}'")

                # Filter the dataframe for matched columns
                df = df[
                    [
                        matched_headers[friendly_name]
                        for friendly_name in matched_headers
                    ]
                ]

                for _, row in df.iterrows():
                    row_data = {"properties": [], "type": "", "mention_text": ""}
                    combined_mention_text = ""
                    parent_type = "invoiceItem"

                    for friendly_name, matched_header in matched_headers.items():
                        cell = row[matched_header]
                        mention_text = cell.get("text")
                        text_anchor = cell.get("text_anchor")
                        page_anchor = cell.get("page_anchor")

                        if (
                            mention_text is None
                            or text_anchor is None
                            or page_anchor is None
                        ):
                            continue

                        # Handling TextAnchor with multiple text segments
                        text_segments = []
                        for segment in text_anchor.text_segments:
                            text_segments.append(
                                {
                                    "start_index": segment.start_index,
                                    "end_index": segment.end_index,
                                }
                            )

                        # Nesting text_segments under text_anchor
                        text_anchor_dict = {"text_segments": text_segments}

                        child_entity_type = friendly_name
                        child_entity = {
                            "type": child_entity_type,
                            "mention_text": mention_text,
                            "text_anchor": text_anchor_dict,  # Using the nested text_anchor structure
                        }

                        vertices = [
                            {
                                "x": page_anchor["topLeft"]["x"],
                                "y": page_anchor["topLeft"]["y"],
                            },
                            {
                                "x": page_anchor["bottomRight"]["x"],
                                "y": page_anchor["topLeft"]["y"],
                            },
                            {
                                "x": page_anchor["bottomRight"]["x"],
                                "y": page_anchor["bottomRight"]["y"],
                            },
                            {
                                "x": page_anchor["topLeft"]["x"],
                                "y": page_anchor["bottomRight"]["y"],
                            },
                        ]
                        child_entity["page_anchor"] = {
                            "page_refs": [
                                {
                                    "bounding_poly": {"normalized_vertices": vertices},
                                    "page": str(page_number),
                                }
                            ]
                        }
                        combined_mention_text += mention_text + " "
                        row_data["properties"].append(child_entity)

                    row_data["type"] = parent_type
                    row_data["mention_text"] = combined_mention_text
                    result.append(row_data)

                # display(df)

    # print(result)
    output_blob_name = output_prefix + json_file.split("/")[-1]

    # Convert the JSON string back to a dictionary
    json_data = json.loads(documentai.Document.to_json(document))

    # Append the serializable result to the 'entities' field
    json_data["entities"].extend(result)

    # Convert the modified JSON data back to a string
    json_string = json.dumps(json_data, indent=4)

    # Call the store_document_as_json function
    utilities.store_document_as_json(json_string, output_bucket_name, output_blob_name)
    # break

print("DONE")

## **Output** 

### Input Form Parser Output Json 

<img src="./images/input.png" width=600 height=600 alt="None">


### Output Table to line item entity converted Json

The output JSON will contain data extracted from Form parser tables present in the source document, and this data will be structured as line items. The extraction and structuring process will be guided by the specifications provided in the user_input dictionary. The user_input dictionary serves as a blueprint: it maps specific headers (as they appear in the source document) to corresponding entity names (as they should be represented in the output JSON). By following these mappings, the script can convert table structure into line items in the resulting JSON.

<img src="./images/output.png" width=600 height=600 alt="None">

**Note:**
* Code works for tables with over 7 columns and multiple rows or tables resembling the example shown above.
* When converting tables to line items, the table header becomes part of the line items and gets included in the processed JSON.
* Discrepancies may occur during conversion due to reliance on the form parser table output, resulting in potential merging of columns or rows.